In [125]:
import os

import matplotlib as plt
import pandas as pd

%matplotlib inline
plt.style.use('ggplot')

In [143]:
files = os.listdir('./by-offense/raw')
del files[0]
files

['jtdc-offense-type-arrest-warrant.csv',
 'jtdc-offense-type-court-hold.csv',
 'jtdc-offense-type-new-offense.csv',
 'jtdc-offense-type-probation-viol.csv',
 'jtdc-offense-type-total.csv']

In [144]:
pd.read_csv('./by-offense/raw/%s' % files[0], header=None, skiprows=1)

,0,1,2,3
0,Juvenile Arrest Warrant,23 25% 53,60 45% 38,83 37% 42
1,Person,9 39% 26,25 42% 47,34 41% 42
2,Weapons,8 35% 42,11 18% 51,19 23% 47
3,Property,2 9% 230,18 30% 17,20 24% 38
4,Drug - Controlled Substance,3 13% 58,2 3% 16,5 6% 41
5,Drug – Cannabis,0 0% 0,0 0% 0,0 0% 0
6,Sex Crimes,0 0% 0,3 5% 29,3 4% 29
7,Other,1 4% 11,1 2% 99,2 2% 55


In [164]:
def split_cols(x):
    for i in range(1, 4):
        if i == 1:
            s = 'no_probation'
        elif i == 2:
            s = 'probation'
        else:
            s = 'total'
        x['%s_num' % s] = x[i].apply(lambda x: x.split(' ')[0])
        x['%s_pct' % s] = x[i].apply(lambda x: x.split(' ')[1]).str.rstrip('%')
        x['%s_avg_los' % s] = x[i].apply(lambda x: x.split(' ')[2])
    x.columns.values[0] = 'offense_type'
    x['offense_type'] = x['offense_type'].str.rstrip()
    x.drop([1, 2, 3], axis=1, inplace=True)

In [165]:
# generate individual csvs for each reason for detention by offense type
for file in files:
    df = pd.read_csv('./by-offense/raw/%s' % file, header=None, skiprows=1)
    split_cols(df)
    df.to_csv('./by-offense/processed/%s' % file, index=False)

In [196]:
# read in individual csvs
df = pd.concat([pd.read_csv('by-offense/processed/%s' % f) for f in files]).reset_index(drop=True)
# grab reasons for detention (for each reason, total row precedes offense type breakdown)
offense_types = list(df.ix[[n for n in range(0, len(df), 8)]]['offense_type'])
# drop grand total row (bc we can reproduce if we want)
df.drop(len(df)-1, inplace=True)
# set reason_for_detention for each range of rows it applies to and delete total row
for ind, x in enumerate(range(0, len(df), 8)):
    df.loc[x:x+8, 'reason_for_detention'] = offense_types[ind]
    if x != 24: # probation violation; only one row
        df.drop(x, inplace=True)
df.ix[:, [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

,reason_for_detention,offense_type,no_probation_num,no_probation_pct,no_probation_avg_los,probation_num,probation_pct,probation_avg_los,total_num,total_pct,total_avg_los,reason_for_detention
1,Juvenile Arrest Warrant,Person,9,39,26,25,42,47,34,41,42,Juvenile Arrest Warrant
2,Juvenile Arrest Warrant,Weapons,8,35,42,11,18,51,19,23,47,Juvenile Arrest Warrant
3,Juvenile Arrest Warrant,Property,2,9,230,18,30,17,20,24,38,Juvenile Arrest Warrant
4,Juvenile Arrest Warrant,Drug - Controlled Substance,3,13,58,2,3,16,5,6,41,Juvenile Arrest Warrant
5,Juvenile Arrest Warrant,Drug – Cannabis,0,0,0,0,0,0,0,0,0,Juvenile Arrest Warrant
6,Juvenile Arrest Warrant,Sex Crimes,0,0,0,3,5,29,3,4,29,Juvenile Arrest Warrant
7,Juvenile Arrest Warrant,Other,1,4,11,1,2,99,2,2,55,Juvenile Arrest Warrant
9,Court Hold,Person,5,56,13,4,36,16,9,45,14,Court Hold
10,Court Hold,Weapons,3,33,25,3,27,22,6,30,24,Court Hold
11,Court Hold,Property,1,11,49,4,36,16,5,25,22,Court Hold


In [186]:
df.to_csv('by-offense/processed/jtdc-offense-type-all.csv', index=False)